In [ ]:
# validating manual counting of whole ortho mosaic



"""
#main steps


# change csv file into yolo 

yolo is in proportion of pixels, while csv is in pixel numbers, i assume starting top left

csv_x / width, y/height

# currenty testing a bounding box of 150  pixels
w = box_size/29184
h = box_size/11552



# add small pixel bounding boxes
# take centroid output and so the same small bounding boxes

# tiles and subset into test set

# validate 


"""



In [33]:

#imports 
import os

# first check the wd is not notebooks but the main folder
print("cwd is", os.getcwd())

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
    print("cwd changed to", os.getcwd())


# set paths

manual_count_dir = "datasets/manual_ortho_count/counts"
count_csv = "datasets/manual_ortho_count/points_1259.csv" # csv of points
predict_points = "datasets/export_predictions/final/labels/val/ortho_clip_23april.txt"


cwd is /home/tmueller/github/AIggregation


In [ ]:
# read in manual count csv

import pandas as pd
import shutil

width = 29184
height = 11552
bounding_box_size = 150


# read in point csv
data = pd.read_csv(count_csv)

data = data.drop(['survey id', 'image'], axis=1)# drop useless columns
data['class'] = 0 # set class to zero to match yolo

# set width and height of bounding box
data['w'] = bounding_box_size/ width
data['h'] = bounding_box_size/ height

data['x'] = data['x'] / width
data['y'] = data['y'] / height

data['conf'] = 100

# save csv to labels folder
if not os.path.exists(manual_count_dir):
    os.makedirs(manual_count_dir)
    os.makedirs(os.path.join(manual_count_dir,"images/val/"))
    os.makedirs(os.path.join(manual_count_dir, "labels/val/"))
elif len(os.listdir(manual_count_dir)) > 0:
    raise Exception("Target folder should be empty")

data.to_csv("datasets/manual_ortho_count/counts/labels/val/ortho_clip_23april.txt", sep=' ', index=False, header=False)

# copy ortho to folder to match yolo structure
shutil.copy("datasets/drone_ortho/ortho_clip_23april.png", os.path.join(manual_count_dir,"images/val/"))

# and copy non test list so it creates the same testset
shutil.copy("datasets/testset/nontest.txt", "datasets/manual_ortho/counts")


'datasets/manual_ortho_count/counts/images/val/ortho_clip_23april.png'

In [ ]:
# next tile labels with overwrite=FALSE


# set parameters for tiling script
# -------------------------------------------------------------------------------------------------------------------- #
source_path = manual_count_dir                              # directory where model prediction exported to
target_path = "./datasets/manual_ortho_count/tiles"         # directory to save tiled testset
img_ext = ".png"                                            # type of image predicted on
tile_size = 608                                             # size of tiles in pixels. default 608
all_images = "TRUE"                                         # set to FALSE to only keep images with detections
test_ratio = 0.4                                            # proportion of tiles to keep for testset
overwrite = "FALSE"                                         # set to TRUE to create new testlist. default to pulling from existing list one up from target dir
# -------------------------------------------------------------------------------------------------------------------- #

from PIL import Image 

# set max image pixels to none
# otherwise pillow thinks large images might be a bomb DOS attack 
Image.MAX_IMAGE_PIXELS = None


# run riling script using above parameters
%run scripts/yolo_tile_modified.py -source {source_path} -target {target_path} -ext {img_ext}  -size {tile_size} -ratio {test_ratio} -overwrite {overwrite} -all_images {all_images}


Image: datasets/manual_ortho_count/counts/images/val/ortho_clip_23april.png
[(0, <POLYGON ((-34.447 11164.864, 115.553 11164.864, 115.553 11314.864, -34.447 ...>, 100.0), (0, <POLYGON ((2261.444 11040.947, 2411.444 11040.947, 2411.444 11190.947, 2261....>, 100.0), (0, <POLYGON ((3453.405 11145.9, 3603.405 11145.9, 3603.405 11295.9, 3453.405 11...>, 100.0), (0, <POLYGON ((4028.145 11384.542, 4178.145 11384.542, 4178.145 11534.542, 4028....>, 100.0), (0, <POLYGON ((4094.365 11463.256, 4244.365 11463.256, 4244.365 11613.256, 4094....>, 100.0), (0, <POLYGON ((3612.845 11138.378, 3762.845 11138.378, 3762.845 11288.378, 3612....>, 100.0), (0, <POLYGON ((4204.982 11084.237, 4354.982 11084.237, 4354.982 11234.237, 4204....>, 100.0), (0, <POLYGON ((4348.953 11298.931, 4498.953 11298.931, 4498.953 11448.931, 4348....>, 100.0), (0, <POLYGON ((9905.732 11124.65, 10055.732 11124.65, 10055.732 11274.65, 9905.7...>, 100.0), (0, <POLYGON ((11368.175 11007.2, 11518.175 11007.2, 11518.175 11157.2, 11368

In [ ]:

# below is a section editing ground truth but lets ignore that for now

"""

import glob

# copy test set to a new folder
gt = "datasets/testset/groundtruth_testset"
gt_manual = "datasets/testset/groundtruth_testset_manual"

if not os.path.exists(gt):
    os.makedirs(gt_manual)

shutil.copytree(gt, gt_manual)



label_paths = glob.glob(f'{gt_manual}/labels/*')


label_paths

# change ground truth label bounding box size 
for file in label_paths:
    print(file)
    data = pd.read_csv(file, sep= ' ')
    
    data.iloc[:, 3]= 0.246711 # change w
    data.iloc[:, 4]= 0.246711 # change h

    data.to_csv(file, sep=' ', index=False, header=False)

"""



'datasets/testset/tiled_testset_manual'

In [117]:
# validate with fifty one

import os
import fiftyone as fo
from fiftyone import ViewField as F

# set paths
# -------------------------------------------------------------------------------------------------------------------- #
gt_directory = "datasets/testset/groundtruth_testset"       # Directory with labelstudio groundtruth export extracted. Should contain "images" and "labels" folders
manual_test_directory = "datasets/manual_ortho_count/tiles"     # target folder for tile script ("target_path" in previous notebook)
# ------



# import test tiles into fiftyone
datasettest = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = os.path.join(manual_test_directory, "dataset.yaml"),
    label_field= "prediction"
)

# import groundtruthed tiles into fiftyone
dataset_ground = fo.Dataset.from_dir(
    dataset_type=fo.types.YOLOv5Dataset,
    yaml_path = os.path.join(gt_directory,"dataset.yaml"),
    label_field= "ground_truth"
)

# merge the datasets, merging images/labels with the same file name
key_fcn = lambda sample: os.path.basename(sample.filepath)

datasettest.merge_samples(dataset_ground, key_fcn=key_fcn)



# delete any oblong box predictions
# these are created by splitting tiles through the middle of an existing bounding box. Nest predictions should not be oblong

# Compute the dimensions of each bounding box in pixels
box_width, box_height = F("bounding_box")[2], F("bounding_box")[3]

# remove detections where one side of the box is greater than 2.5 time the other
def get_label_fields(sample_collection):
    """Get the (detection) label fields of a Dataset or DatasetView."""
    label_fields = list(
        sample_collection.get_field_schema(embedded_doc_type=fo.Detections).keys()
    )
    return label_fields


for lf in get_label_fields(datasettest):
    datasettest_temp = datasettest.filter_labels(
    "prediction", (box_height < (box_width*2)) & (box_width < (box_height*2)), only_matches=False)


for lf in get_label_fields(datasettest_temp):
    datasettest_keep = datasettest_temp.filter_labels(
    "ground_truth", (box_height < (box_width*2)) & (box_width < (box_height*2)), only_matches=False)




session = fo.launch_app(datasettest)


 100% |█████████████████| 235/235 [325.0ms elapsed, 0s remaining, 723.0 samples/s]      
 100% |█████████████████| 365/365 [287.0ms elapsed, 0s remaining, 1.3K samples/s]     
Indexing dataset...
 100% |█████████████████| 235/235 [130.2ms elapsed, 0s remaining, 1.8K samples/s]     
Merging samples...
 365 [722.4ms elapsed, ? remaining, 506.3 samples/s] 
